In [182]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
# import xgboost as xgb
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())


0it [00:00, ?it/s]


In [183]:
train = pd.read_csv('./train_1.csv').fillna(0)
key = pd.read_csv('./key_1.csv')

In [184]:
sub = pd.read_csv("./sample_submission_1.csv")

In [185]:
for col in train.columns[1:]:
    train[col] = pd.to_numeric(train[col],downcast='integer')

In [246]:
def get_median(row, last_n_days):
    return pd.rolling_median(row, window = last_n_days, min_periods=7)    

In [245]:
def get_mean(row, last_n_days):
    return pd.rolling_mean(pd.Series(row), window = last_n_days)    

In [257]:
def get_long_stats(func_type, last_n_days, df, df_melt):

    if func_type == "median":
        rolling_stats = df.progress_apply(get_median, last_n_days = last_n_days, axis = 1)
    elif func_type == "mean":
        rolling_stats = df.progress_apply(get_mean, last_n_days = last_n_days, axis = 1)

    rolling_stats_df = rolling_stats.apply(pd.Series)    
    rolling_stats_df.columns = df.columns
    
    rolling_stats_melt = pd.melt(rolling_stats_df, id_vars=['Page'], var_name="date", value_name=func_type + "_" + str(last_n_days))
    
    df_melt = pd.merge(df_melt, rolling_stats_melt, on = ["Page", "date"])     
    return df_melt

In [ ]:
train_melt = pd.melt(train, id_vars=['Page'], var_name="date", value_name="visits")

train_melt = get_long_stats("mean", 7, train, train_melt)
train_melt = get_long_stats("mean", 14, train, train_melt)
train_melt = get_long_stats("mean", 21, train, train_melt)
train_melt = get_long_stats("mean", 30, train, train_melt)
train_melt = get_long_stats("mean", 60, train, train_melt)

  0%|          | 1/145063 [00:05<206:14:35,  5.12s/it]/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=7,center=False).mean()
  
145064it [02:01, 1194.79it/s]                            
  0%|          | 1/145063 [12:15<29631:45:25, 735.37s/it]/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=14,center=False).mean()
  
145064it [14:12, 170.13it/s]                               


In [53]:
def traffic_stats():
    
    train_melt["median_7_days"] = train.apply(get_median, last_n_days = 7, axis = 1)
    
    train_feats["median_7_days"] = train.apply(get_median, last_n_days = 7, axis = 1)
    train_feats["mean_7_days"] = train.apply(get_mean, last_n_days = 7, axis = 1)    

    train_feats["median_14_days"] = train.apply(get_median, last_n_days = 14, axis = 1)
    train_feats["mean_14_days"] = train.apply(get_mean, last_n_days = 14, axis = 1)    

    train_feats["median_21_days"] = train.apply(get_median, last_n_days = 21, axis = 1)
    train_feats["mean_21_days"] = train.apply(get_mean, last_n_days = 21, axis = 1)    

    train_feats["median_30_days"] = train.apply(get_median, last_n_days = 30, axis = 1)
    train_feats["mean_30_days"] = train.apply(get_mean, last_n_days = 30, axis = 1)    

    train_feats["median_60_days"] = train.apply(get_median, last_n_days = 60, axis = 1)
    train_feats["mean_60_days"] = train.apply(get_mean, last_n_days = 60, axis = 1)        
    

In [74]:
train_feats = pd.DataFrame()
train_feats["Page"] = train["Page"]

traffic_stats()

train_feats.to_csv("train_feats.csv", index = False)

In [86]:
key["trunc_page"] = key.apply(lambda x: x["Page"][:-11], axis = 1)
key.to_csv("key_trunc.csv", index = False)

In [87]:
train_feats = pd.read_csv("train_feats.csv")
key = pd.read_csv("key_trunc.csv")

train_merged = pd.merge(train_feats, key, left_on="Page", right_on="trunc_page", how="inner")
train_merged.drop("Page_x", inplace=True, axis = 1)
train_merged.drop("Page_y", inplace=True, axis = 1)

train_merged.to_csv("train_merged.csv", index = False)

In [93]:
train_merged_filtered = train_merged[["Id", "median_60_days"]]

In [111]:
sub = pd.merge(sub, train_merged_filtered, on = "Id")
sub.drop("Visits", inplace=True, axis = 1)
sub.columns = ["Id", "Visits"]

In [112]:
sub.to_csv("median_60_days.csv", index = False)

In [130]:
train.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0


In [134]:
train_melt = pd.melt(train, id_vars=['Page'], var_name="date", value_name="visits")

In [135]:
train_melt.head()

,Page,date,visits
0,2NE1_zh.wikipedia.org_all-access_spider,2015-07-01,18.0
1,2PM_zh.wikipedia.org_all-access_spider,2015-07-01,11.0
2,3C_zh.wikipedia.org_all-access_spider,2015-07-01,1.0
3,4minute_zh.wikipedia.org_all-access_spider,2015-07-01,35.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,2015-07-01,0.0


In [255]:
a = pd.DataFrame([[1,1,1,4,5,6,7,8]])
a.apply(get_median, last_n_days = 3, axis = 1)

0    1
1    1
2    1
3    4
4    5
5    6
6    7
7    8
Name: 0, dtype: int64
0    1
1    1
2    1
3    4
4    5
5    6
6    7
7    8
Name: 0, dtype: int64


/Library/Python/2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pd.rolling_median is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=7,window=3,center=False).median()
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: ('min_periods (7) must be <= window (3)', u'occurred at index 0')

In [ ]:
def run_xgb(x_train, x_test, x_label):

    # x_train = pd.concat([pos_train, neg_train]) #Concat positive and negative
    # y_train = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist() #Putting in 1 and 0

    # x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=4242)

    # Set our parameters for xgboost
    params = {}
    params['objective'] = 'binary:logistic'
    params['eval_metric'] = 'logloss'
    params['eta'] = 0.05
    params['max_depth'] = 6
    params['silent'] = 1

    d_train = xgb.DMatrix(x_train, label=x_label)
    d_test = xgb.DMatrix(x_test)

    watchlist = [(d_train, 'train')]

    bst = xgb.train(params, d_train, 100, watchlist, early_stopping_rounds=50, verbose_eval=50)

    p_test = bst.predict(d_test)

    # xgb.plot_importance(bst)
    # pyplot.show()

    return p_test